In [216]:
import openai
from openai_cost_tracker import query_openai
from openai import OpenAI
import pandas as pd
from nltk import translate
import Levenshtein as lev
import os
from dotenv import load_dotenv

In [217]:
load_dotenv("./api_key")

True

In [220]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get('OPENAI_API_KEY')
)

### Prompting function

In [193]:
def generate_n_shot_prompt(df, transformation_type, num_examples):
    """    
    Parameters:
    - df: A pandas DataFrame with columns 'input', 'transformation', and 'output'.
    - transformation_type: A string indicating the type of transformation to filter for.
    - num_examples: An integer indicating the number of examples to include in the prompt.
    
    Returns:
    - A string containing the generated prompt.
    """
    # Filter the DataFrame for the specific transformation type
    # filtered_df = df[df['Transformation'] == transformation_type]
    
    # # Check if there are enough examples
    # if num_examples > len(filtered_df):
    #     raise ValueError("The number of requested examples exceeds the available rows for the specified transformation type.")

    filtered_df = df[df['Transformation'] == transformation_type].sample(frac=1).reset_index(drop=True)
    
    # Ensure there are enough examples
    if num_examples + 1 > len(filtered_df):
        raise ValueError("Not enough examples of the requested transformation type in the DataFrame.")
    
    # Randomly select num_examples rows
    selected_examples = filtered_df.head(num_examples)
    
    prompt = f"Transformation: {transformation_type}\n\n"
    
    # Add each example to the prompt
    for i, row in enumerate(selected_examples.itertuples(), 1):
        prompt += f"{i}. Original: \"{row.Input}\" Transformed: \"{row.Output}\"\n"
    
    # Add instruction for the next input
    prompt += "\nGiven the following input, apply the same transformation:\n\n"
    query_example = filtered_df.iloc[num_examples]
    prompt += f"Input: \"{query_example.Input}\"\nOutput: "
    # print(prompt)
    
    return prompt, query_example

### Evaluation functions

In [138]:
def compute_exact_string_match(actual, expected):
    """
    Compares two strings to check if they are exactly the same.

    Parameters:
    - actual: String. The actual output from the transformation or prediction.
    - expected: String. The expected correct output.

    Returns:
    - A boolean value. True if the actual output matches the expected output exactly, False otherwise.
    """
    return actual == expected

In [122]:
def calculate_scores(actual, expected):
    is_exact_match = compute_exact_string_match(actual, expected)
    levenshtein = lev.distance(actual, expected)
    chrF = translate.chrf_score.sentence_chrf(actual, expected)
    return {
        "is_exact_match": is_exact_match,
        "Levenshtein": levenshtein, 
        "chrF": chrF
    }

### Querying

In [160]:
def query_with_cost(model, prompt, max_tokens, cost_per_1000_tokens=0.02):
    """
    A wrapper function to call OpenAI's Completion API and print the cost of the query.

    Parameters:
    - model (str): The model to use for the completion.
    - prompt (str): The prompt to pass to the model.
    - max_tokens (int): The maximum number of tokens to generate in the completion.
    - cost_per_1000_tokens (float): The cost per 1,000 tokens for your OpenAI plan.

    Prints the cost of the query in the specified format.
    """
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=model,
    )
    usage_info = response.usage

    input_tokens = len(prompt.split())  # Simplified estimation of input tokens
    output_tokens = usage_info.total_tokens - input_tokens  # Estimating output tokens
    total_tokens = usage_info.total_tokens  # Total tokens

    cost = (total_tokens / 1000) * cost_per_1000_tokens  # Calculate cost

    print(f"Input tokens: {input_tokens} | Output tokens: {output_tokens} | Cost: ${cost:.4f} | Total: ${cost:.4f}")

    output = response.choices[0].message.content
    print("Prompt:")
    print(prompt + output)
    # print(response.choices)
    # print(response.choices[0].message.content)
    return output

# Example usage
model = "gpt-3.5-turbo"
prompt = "What is the capital of France?"
max_tokens = 50

# Adjust the cost_per_1000_tokens according to your actual plan
query_with_cost(model, prompt, max_tokens, cost_per_1000_tokens=0.02)


Input tokens: 6 | Output tokens: 15 | Cost: $0.0004 | Total: $0.0004
Prompt:
What is the capital of France?The capital of France is Paris.


'The capital of France is Paris.'

In [166]:
def query_with_cost_and_compare(model, prompt, max_tokens, expected, cost_per_1000_tokens=0.02):
    actual = query_with_cost(model, prompt, max_tokens)
    print("\n-------")
    print("Actual: ", actual, "Expected ", expected)
    # print("Expected:", expected)
    return calculate_scores(actual, expected)


#### Data Parsing

In [136]:
pilot_bribri_train = './americasnlp2024/ST2_EducationalMaterials/pilotdata/bribri-train.tsv'
pilot_bribri_train_df = pd.read_csv(pilot_bribri_train, delimiter='\t', names=['ID', 'Input', 'Transformation', 'Output'])

In [68]:
pilot_bribri_train_df

,ID,Input,Transformation,Output
0,Bribri001,Ie’ tso bua’,TYPE:NEG,Ie’ kë̀ ku̠ bua’
1,Bribri002,Ie’ tso bua’,"TENSE:PER_REM, ASPECT:PER",Ie’ bák bua’
2,Bribri003,Ie’ tso bua’,"TYPE:NEG, TENSE:PER_REM, ASPECT:PER",Ie’ kë̀ ku̠’bak bua’
3,Bribri004,Ie’ tso bua’,TYPE:NEG,Kë̀ i ku̠’ bua’
4,Bribri005,Ye’ i̠ni̠’,"TENSE:IMP_REC, ASPECT:IMP",Ie’ i̠nù̠
...,...,...,...,...
114,Bribri115,Ye’ e̠’ tkë’rë,"TENSE:IMP_REC, ASPECT:IMP",Ye’ e̠’ tkö̀rö
115,Bribri116,Ye’ e̠’ tkë’rë,"MODE:IMP, TENSE:IMP_REC, ASPECT:IMP",Be’ e̠’ tkö́rö
116,Bribri117,Pûs kapë’wa̠,NUMBER:PL,Pûs kapë’ulur
117,Bribri118,Ye’ tö skuë̀ kö́twa̠,NUMBER:PL,Ye’ tö skuë̀ kö́tulur


In [98]:
pilot_bribri_train_df[pilot_bribri_train_df['Transformation'] == 'TYPE:NEG']

,ID,Input,Transformation,Output
0,Bribri001,Ie’ tso bua’,TYPE:NEG,Ie’ kë̀ ku̠ bua’
3,Bribri004,Ie’ tso bua’,TYPE:NEG,Kë̀ i ku̠’ bua’
32,Bribri033,Ye’ tö ù sú̠,TYPE:NEG,Ye’ kë̀ wa̠ ù sù̠ne̠
60,Bribri061,Ye’ stsë’,TYPE:NEG,Ye’ kë̀ stsë̀ne̠
79,Bribri080,Ye’ é̠n a̠ àrros ña̠’ku,TYPE:NEG,Kë̀ ye’ é̠n a̠ àrros ña̠’ku
109,Bribri110,Ye’ wa̠ sawë́ chka katárule,TYPE:NEG,Ye’ kë̀ wa̠ sawë́ chka katárule


In [184]:
pilot_bribri_train_df.groupby(['Transformation'])['Transformation'].count()

Transformation
ASPECT:INC                                                   1
MODE:ADV                                                     1
MODE:ADV, TENSE:PER_REM, ASPECT:PER, VOICE:MID               1
MODE:EXH, TENSE:IMP_HAB, ASPECT:IMP                          1
MODE:EXH, TENSE:IMP_HAB, ASPECT:IMP, NUMBER:PL               1
MODE:IMP, TENSE:IMP_REC, ASPECT:IMP                          6
MODE:IND, TENSE:FUT_PT, ASPECT:IMP, VOICE:MID                1
MODE:IND, TENSE:IMP_HAB, ASPECT:IMP, VOICE:MID               2
MODE:OPT, TENSE:IMP_HAB, ASPECT:IMP                          2
MODE:OPT, TENSE:IMP_HAB, VOICE:MID                           1
NUMBER:PL                                                    4
TENSE:FUT_CER, ASPECT:IMP                                    4
TENSE:FUT_CER, ASPECT:IMP, VOICE:MID                         1
TENSE:FUT_PT, ASPECT:IMP                                     4
TENSE:IMP_CON, ASPECT:IMP                                    3
TENSE:IMP_HAB, ASPECT:IMP               

In [182]:
two_shot_negation_prompt, query_example = generate_n_shot_prompt(pilot_bribri_train_df, 'TYPE:NEG', 2)

In [181]:
print(generate_n_shot_prompt(pilot_bribri_train_df, 'TYPE:NEG', 2))

('Transformation: TYPE:NEG\n\n1. Original: "Ie’ tso bua’" Transformed: "Ie’ kë̀ ku̠ bua’"\n2. Original: "Ie’ tso bua’" Transformed: "Kë̀ i ku̠’ bua’"\n\nGiven the following input, apply the same transformation:\n\nInput: "Ye’ tö ù sú̠"\nOutput: ', ID                             Bribri033
Input                      Ye’ tö ù sú̠
Transformation                  TYPE:NEG
Output            Ye’ kë̀ wa̠ ù sù̠ne̠
Name: 32, dtype: object)


In [190]:
query_with_cost_and_compare(model, two_shot_negation_prompt, 1000, query_example.Output)

Input tokens: 36 | Output tokens: 78 | Cost: $0.0023 | Total: $0.0023
Prompt:
Transformation: TYPE:NEG

1. Original: "Ie’ tso bua’" Transformed: "Ie’ kë̀ ku̠ bua’"
2. Original: "Ie’ tso bua’" Transformed: "Kë̀ i ku̠’ bua’"

Given the following input, apply the same transformation:

Input: "Ye’ tö ù sú̠"
Output: "Ye’ kë̀ tö ù sú̠"

-------
Actual:  "Ye’ kë̀ tö ù sú̠" Expected  Ye’ kë̀ wa̠ ù sù̠ne̠


{'is_exact_match': False, 'Levenshtein': 9, 'chrF': 0.3395043464140041}

In [185]:
six_shot_imperfect = generate_n_shot_prompt(pilot_bribri_train_df, 'TENSE:IMP_REC, ASPECT:IMP', 6)

In [187]:
query_with_cost_and_compare(model, six_shot_imperfect[0], 1000, six_shot_imperfect[1].Output)

Input tokens: 60 | Output tokens: 163 | Cost: $0.0045 | Total: $0.0045
Prompt:
Transformation: TENSE:IMP_REC, ASPECT:IMP

1. Original: "Ye’ i̠ni̠’" Transformed: "Ie’ i̠nù̠"
2. Original: "Ye’ i̠ni̠’" Transformed: "Ye’ i̠nù̠"
3. Original: "Ye’ i̠ni̠’" Transformed: "Ye’ tso i̠nú̠k"
4. Original: "Ye’r i sú̠" Transformed: "Ye’r i sa̠wè̠"
5. Original: "Ye’ tú̠ne̠’" Transformed: "Ye’ tú̠n"
6. Original: "Ye’ tú̠ne̠’" Transformed: "I tú̠n"

Given the following input, apply the same transformation:

Input: "Ye’ stsë’"
Output: "Ye’ stsë̠̀"

-------
Actual:  "Ye’ stsë̠̀" Expected  Ie’ stsö̀ bua’


{'is_exact_match': False, 'Levenshtein': 8, 'chrF': 0.2756681874734525}

In [194]:
six_shot_imperfect = generate_n_shot_prompt(pilot_bribri_train_df, 'TENSE:IMP_REC, ASPECT:IMP', 6)

In [221]:
query_with_cost_and_compare(model, six_shot_imperfect[0], 1000, six_shot_imperfect[1].Output)

Input tokens: 63 | Output tokens: 152 | Cost: $0.0043 | Total: $0.0043
Prompt:
Transformation: TENSE:IMP_REC, ASPECT:IMP

1. Original: "Ye’ i̠ni̠’" Transformed: "Ye’ tso i̠nú̠k"
2. Original: "Ye’ stsë’" Transformed: "Ye’ stsö̀"
3. Original: "Ye’ tú̠ne̠’" Transformed: "I tú̠n"
4. Original: "Ye’ i̠ni̠’" Transformed: "Ie’ i̠nù̠"
5. Original: "Ye’ tö i kíme̠’" Transformed: "Ye’ tö i kimë̀"
6. Original: "Ye’ tú̠ne̠’" Transformed: "Ye’ tú̠n"

Given the following input, apply the same transformation:

Input: "Ye’r i sú̠"
Output: Ye’r ti suò̠

-------
Actual:  Ye’r ti suò̠ Expected  Ye’r i sa̠wè̠


{'is_exact_match': False, 'Levenshtein': 6, 'chrF': 0.24333288909746562}